# Model: Anane et al. acetate cycling 2017

###### This file has been automatically generated with the Template Generator
###### Here is the link to the GitHub repository: https://git.tu-berlin.de/bvt-htbd/modelling/e-coli-model-overview/
###### Version 1.7.2 from 30.09.25 Matteo Di Fiore, New Feature: Better log messages
###### Name of the Paper: Modelling overflow metabolism in Escherichia coli by acetate cycling
###### DOI: https://doi.org/10.14279/depositonce-8272
###### Description and assumptions: This is the model for overflow metabolism of E. coli as described in the paper of Anane et al. 2017. The model includes substrate uptake, acetate production and consumption, oxygen uptake, and biomass growth. The model is based on Monod kinetics with inhibition terms for acetate and oxygen. The model can simulate both batch and fed-batch cultures, with the option to include a feeding strategy based on substrate concentration. The model also includes a maintenance term for biomass growth. The model is designed to run without if clauses to see if acetate is produced or consumed. It happens at the same time.
###### Goal of Simulation: Comparison with the Paper
###### Created on 2025 Jul Fri
###### Updated on 08.10.2025
###### @authors: Matteo Di Fiore

In [1]:
# Used packages
import numpy as np                                         # Numerical computing, arrays, linear algebra
from scipy.integrate import odeint                         # ODE solver for integrating differential equations
from math import exp                                       # Exponential function (for kinetics and growth rates)
import class_plot as cplt                                  # Custom plotting class (data visualization)
import class_fedbatch as cfb                               # Custom fed-batch model class (feeding & pulse logic)
import class_allthedata as catd                            # Custom all-the-data model class (dictionary creation)
from markdown_generation import generate_markdown_table    # Custom markdown generation function
from IPython.display import Markdown                       # Display formatted Markdown in Jupyter notebooks
import os                                                  # System coding (to save folders)

In [2]:
# Time variables
t0 = 0.0 # initial time of simulation in h
dt = 0.01 # steps of simulation in h
t_end = 20 # change if you want, this is the time of the end of simulation in h
t_constant_feed = 12 # change if you want, this is the time of the beginning of continuos feeding in h
t_fedbatch_start = 6 # change if you want, this is the time of the beginning of fedbatch in h
t = np.linspace(t0, t_end, int(t_end/dt)+1)

In [3]:
# Feed options
# this value decides when the feeding starts based on the remaining Substrate
substrate_limit_fedbatch = 0.01    # this value is in (g/L) change the value if you want

# initializing the feeding class
# you can call the feeding class with t_fedbatch_start, substrate_limit_fedbatch, t_constant_feed and any combination of them
# just remember that if you use both t_fedbatch_start and substrate_limit_fedbatch the feeding will start when the t_fedbatch_start is reached
# if you want a fedbatch (with the feed start happening at t_fedbatch) it should be like this: feed = cfb.Feed(t_fedbatch_start = t_fedbatch_start), just remember to define t_fedbatch_start with a number
# if you want a fedbatch (with the feed start happening at substrate_limit_fedbatch) it should be like this: feed = cfb.Feed(substrate_limit = substrate_limit_fedbatch), just remember to attribute a number to substrate_limit_fedbatch (you can find it at the button of this code block)
# if you want a fedbatch (with the feed start happening at t_fedbatch or substrate_limit_fedbatch) and a constant feed it should be like this: feed = cfb.Feed(t_const = t_constant_feed, t_fedbatch_start = t_fedbatch_start) or feed = cfb.Feed(t_const = t_constant_feed, substrate_limit = substrate_limit_fedbatch) , just remember to define teh variables with a number
feed = cfb.Feed() # write here the variable which determine how you feed. In this case it is a batch simulation

In [4]:
variables = dict(
    Biomass = dict(
        description = 'Biomass',
        unit = '[g/L]',
        initial_value = 0.17,
        boundaries = [None, None],
        estimable = None,
        weight = None,
        plotting = {'plot': None, 'range_fedbatch': None},
        volume_related = None,
        conversion_factor = None),
    Substrate = dict(
        description = 'extracellular concentration of Substrate',
        unit = '[g/L]',
        initial_value = 4.94,
        boundaries = [None, None],
        estimable = None,
        weight = None,
        plotting = {'plot': None, 'range_fedbatch': None},
        volume_related = None,
        conversion_factor = None),
    Acetate = dict(
        description = 'Acetate',
        unit = '[g/L]',
        initial_value = 0.0129,
        boundaries = [None, None],
        estimable = None,
        weight = None,
        plotting = {'plot': None, 'range_fedbatch': None},
        volume_related = None,
        conversion_factor = None),
    Volume = dict(
        description = 'Volume of the Medium in the Reactor',
        unit = '[L]',
        initial_value = 2,
        boundaries = [None, None],
        estimable = None,
        weight = None,
        plotting = {'plot': None, 'range_fedbatch': None},
        volume_related = None,
        conversion_factor = None),
    DOTa = dict(
        description = 'actual dissolved oxygen tension',
        unit = '[%]',
        initial_value = 98,
        boundaries = [None, None],
        estimable = None,
        weight = None,
        plotting = {'plot': None, 'range_fedbatch': None},
        volume_related = None,
        conversion_factor = None),
    DOT = dict(
        description = 'measured dissolved oxygen tension',
        unit = '[%]',
        initial_value = 98,
        boundaries = [None, None],
        estimable = None,
        weight = None,
        plotting = {'plot': None, 'range_fedbatch': None},
        volume_related = None,
        conversion_factor = None),
)

In [5]:
# For the Variables
md_output = generate_markdown_table(variables, 'Variables')
# Display in Jupyter
Markdown(md_output)

## Variables

| name | Lower boundary | Upper boundary | conversion_factor | description | estimable | initial_value | unit | volume_related | weight |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Biomass | None | None | None | Biomass | None | 0.17 | [g/L] | None | None |
| Substrate | None | None | None | extracellular concentration of Substrate | None | 4.94 | [g/L] | None | None |
| Acetate | None | None | None | Acetate | None | 0.0129 | [g/L] | None | None |
| Volume | None | None | None | Volume of the Medium in the Reactor | None | 2 | [L] | None | None |
| DOTa | None | None | None | actual dissolved oxygen tension | None | 98 | [%] | None | None |
| DOT | None | None | None | measured dissolved oxygen tension | None | 98 | [%] | None | None |

## Variables - plotting specs

| Name | plot | range_fedbatch |
| --- | --- | --- |
| Biomass | None | None |
| Substrate | None | None |
| Acetate | None | None |
| Volume | None | None |
| DOTa | None | None |
| DOT | None | None |


In [6]:
inputs = dict(
    Feed = dict(
        description = 'Feed',
        unit = '[L/h]',
        concentration = 300,
        plotting = {'plot': None, 'stem': None, 'range': [None, None]}),
)

In [7]:
# For the Inputs
md_output = generate_markdown_table(inputs, 'Inputs')
# Display in Jupyter
Markdown(md_output)

## Inputs

| name | concentration | description | unit |
| --- | --- | --- | --- |
| Feed | 300 | Feed | [L/h] |

## Inputs - plotting specs

| Name | plot | range | stem |
| --- | --- | --- | --- |
| Feed | None | [None, None] | None |


In [8]:
parameters = dict(
    K_ap = dict(value = 0.5088, min = None, max = None, vary = None, unit = '[g/L]', description = 'Monod-type saturation constant, intracellular acetate production'),
    K_sa = dict(value = 0.0128, min = None, max = None, vary = None, unit = '[g/L]', description = 'affinity constant, acetate consumption'),
    K_o = dict(value = 0.0001, min = None, max = None, vary = None, unit = '[g/L]', description = 'affinity constant, oxygen consumption'),
    K_s = dict(value = 0.0381, min = None, max = None, vary = None, unit = '[g/L]', description = 'affinity constant, substrate consumption'),
    K_ia = dict(value = 1.2602, min = None, max = None, vary = None, unit = '[g/L]', description = 'inhibition constant, inhibition of glucose uptake by acetate'),
    K_is = dict(value = 1.8383, min = None, max = None, vary = None, unit = '[g/L]', description = 'inhibition constant, inhibition of acetate uptake by glucose'),
    p_a_max = dict(value = 0.2286, min = None, max = None, vary = None, unit = '[g/g/h]', description = 'max specific acetate production rate'),
    q_a_max = dict(value = 0.1148, min = None, max = None, vary = None, unit = '[g/g/h]', description = 'max specific acetate consumption rate'),
    q_m = dict(value = 0.0133, min = None, max = None, vary = None, unit = '[g/g/h]', description = 'specific maintenance coefficient'),
    q_s_max = dict(value = 0.635, min = None, max = None, vary = None, unit = '[g/g/h]', description = 'max spec glucose uptake rate'),
    Y_as = dict(value = 0.8938, min = None, max = None, vary = None, unit = '[g/g]', description = 'Yield coefficient of Acetate on substrate'),
    Y_oa = dict(value = 0.5221, min = None, max = None, vary = None, unit = '[g/g]', description = 'Yield coefficient of oxygen on Acetate'),
    Y_xa = dict(value = 0.5794, min = None, max = None, vary = None, unit = '[g/g]', description = 'Yield coefficient of biomass on Acetate'),
    Y_em = dict(value = 0.5321, min = None, max = None, vary = None, unit = '[g/g]', description = 'Yield coefficient of biomass exclusive of maintenance'),
    Y_os = dict(value = 1.5722, min = None, max = None, vary = None, unit = '[g/g]', description = 'Yield coefficient of oxygen on Substrate'),
    Y_xsof = dict(value = 0.229, min = None, max = None, vary = None, unit = '[g/g]', description = 'yield of biomass on substrate from auxiliary overflow routes, such as the mixed acid and pentose-phosphate pathways'),
)

In [9]:
# For the Parameters
md_output = generate_markdown_table(parameters, 'Parameters')
# Display in Jupyter
Markdown(md_output)

## Parameters

| name | description | max | min | unit | value | vary |
| --- | --- | --- | --- | --- | --- | --- |
| K_ap | Monod-type saturation constant, intracellular acetate production | None | None | [g/L] | 0.5088 | None |
| K_sa | affinity constant, acetate consumption | None | None | [g/L] | 0.0128 | None |
| K_o | affinity constant, oxygen consumption | None | None | [g/L] | 0.0001 | None |
| K_s | affinity constant, substrate consumption | None | None | [g/L] | 0.0381 | None |
| K_ia | inhibition constant, inhibition of glucose uptake by acetate | None | None | [g/L] | 1.2602 | None |
| K_is | inhibition constant, inhibition of acetate uptake by glucose | None | None | [g/L] | 1.8383 | None |
| p_a_max | max specific acetate production rate | None | None | [g/g/h] | 0.2286 | None |
| q_a_max | max specific acetate consumption rate | None | None | [g/g/h] | 0.1148 | None |
| q_m | specific maintenance coefficient | None | None | [g/g/h] | 0.0133 | None |
| q_s_max | max spec glucose uptake rate | None | None | [g/g/h] | 0.635 | None |
| Y_as | Yield coefficient of Acetate on substrate | None | None | [g/g] | 0.8938 | None |
| Y_oa | Yield coefficient of oxygen on Acetate | None | None | [g/g] | 0.5221 | None |
| Y_xa | Yield coefficient of biomass on Acetate | None | None | [g/g] | 0.5794 | None |
| Y_em | Yield coefficient of biomass exclusive of maintenance | None | None | [g/g] | 0.5321 | None |
| Y_os | Yield coefficient of oxygen on Substrate | None | None | [g/g] | 1.5722 | None |
| Y_xsof | yield of biomass on substrate from auxiliary overflow routes, such as the mixed acid and pentose-phosphate pathways | None | None | [g/g] | 0.229 | None |


In [10]:
constants = dict(
    K_LA = dict(value=220, unit='[1/h]', description='volumetric mass transfer coefficient'),
    DOT_star = dict(value=99, unit='[%]', description='saturation value of dissolved oxygen in the medium'),
    H = dict(value=14000, unit='[Pa/(mol/m3)]', description='Henry’s law coefficient for oxygen at 37°C'),
    tau = dict(value=35, unit='[s]', description='probe response time'),
    C_s = dict(value = 0.391, unit = '[mol C/g]', description = 'Carbon content in substrate'),
    C_x = dict(value = 0.488, unit = '[mol C/g]', description = 'Carbon content in biomass'),
    mu_set = dict(value = 0.222, unit = '[1/h]', description = 'Set specific growth rate'),
    Kp = dict(value = ((1/35)*3600), unit = '[1/s]', description = 'static gain of the sensor given as the inverse of the probe response time'),

)

In [11]:
# For the Constants
md_output = generate_markdown_table(constants, 'Constants')
# Display in Jupyter
Markdown(md_output)

## Constants

| name | description | unit | value |
| --- | --- | --- | --- |
| K_LA | volumetric mass transfer coefficient | [1/h] | 220 |
| DOT_star | saturation value of dissolved oxygen in the medium | [%] | 99 |
| H | Henry’s law coefficient for oxygen at 37°C | [Pa/(mol/m3)] | 14000 |
| tau | probe response time | [s] | 35 |
| C_s | Carbon content in substrate | [mol C/g] | 0.391 |
| C_x | Carbon content in biomass | [mol C/g] | 0.488 |
| mu_set | Set specific growth rate | [1/h] | 0.222 |
| Kp | static gain of the sensor given as the inverse of the probe response time | [1/s] | 102.85714285714285 |


In [12]:
rates = dict(
    q_s = dict(unit = '[g/g/h]', description = 'specific substrate uptake rate'),
    q_s_of = dict(unit = '[g/g/h]', description = 'specific substrate uptake rate for overflow'),
    q_s_ox = dict(unit = '[g/g/h]', description = 'specific substrate uptake rate for oxygen'),
    q_s_ox_an=dict(unit='[g/g/h]', description='specific substrate uptake rate for anabolic part of oxidative metabolism'),
    p_a = dict(unit = '[g/g/h]', description = 'specific acetate production rate'),
    q_sa = dict(unit = '[g/g/h]', description = 'specific acetate consumption rate'),
    q_a = dict(unit = '[g/g/h]', description = 'specific acetate balance rate'),
    q_o = dict(unit = '[g/g/h]', description = 'specific oxygen uptake rate'),
    my = dict(unit = '[1/h]', description = 'specific growth rate'),
)

In [13]:
# For the Rates
md_output = generate_markdown_table(rates, 'Rates')
# Display in Jupyter
Markdown(md_output)

## Rates

| name | description | unit |
| --- | --- | --- |
| q_s | specific substrate uptake rate | [g/g/h] |
| q_s_of | specific substrate uptake rate for overflow | [g/g/h] |
| q_s_ox | specific substrate uptake rate for oxygen | [g/g/h] |
| q_s_ox_an | specific substrate uptake rate for anabolic part of oxidative metabolism | [g/g/h] |
| p_a | specific acetate production rate | [g/g/h] |
| q_sa | specific acetate consumption rate | [g/g/h] |
| q_a | specific acetate balance rate | [g/g/h] |
| q_o | specific oxygen uptake rate | [g/g/h] |
| my | specific growth rate | [1/h] |


In [14]:
# Equations
def algebra(x, t):


    '''------------------- generated variables -------------------'''

    X = x[0]    # Biomass
    S = x[1]    # extracellular concentration of Substrate
    A = x[2]    # Acetate
    V = x[3]    # Volume of the Medium in the Reactor
    DOTa = x[4]    # actual dissolved oxygen tension
    DOT = x[5]    # measured dissolved oxygen tension

    '''------------------- generated parameters -------------------'''

    K_ap = parameters['K_ap']['value']    # Monod-type saturation constant, intracellular acetate production
    K_sa = parameters['K_sa']['value']    # affinity constant, acetate consumption
    K_o = parameters['K_o']['value']    # affinity constant, oxygen consumption
    K_s = parameters['K_s']['value']    # affinity constant, substrate consumption
    K_ia = parameters['K_ia']['value']    # inhibition constant, inhibition of glucose uptake by acetate
    K_is = parameters['K_is']['value']    # inhibition constant, inhibition of acetate uptake by glucose
    p_a_max = parameters['p_a_max']['value']    # max specific acetate production rate
    q_a_max = parameters['q_a_max']['value']    # max specific acetate consumption rate
    q_m = parameters['q_m']['value']    # specific maintenance coefficient
    q_s_max = parameters['q_s_max']['value']    # max spec glucose uptake rate
    Y_as = parameters['Y_as']['value']    # Yield coefficient of Acetate on substrate
    Y_oa = parameters['Y_oa']['value']    # Yield coefficient of oxygen on Acetate
    Y_xa = parameters['Y_xa']['value']    # Yield coefficient of biomass on Acetate
    Y_em = parameters['Y_em']['value']    # Yield coefficient of biomass exclusive of maintenance
    Y_os = parameters['Y_os']['value']    # Yield coefficient of oxygen on Substrate
    Y_xsof = parameters['Y_xsof']['value']    # yield of biomass on substrate from auxiliary overflow routes, such as the mixed acid and pentose-phosphate pathways

    '''------------------- generated constants -------------------'''

    K_LA = constants['K_LA']['value']    # volumetric mass transfer coefficient
    DOT_star = constants['DOT_star']['value']    # saturation value of dissolved oxygen in the medium
    H = constants['H']['value']    # Henry’s law coefficient
    tau = constants['tau']['value']    # probe response time
    C_s = constants['C_s']['value']    # Carbon content in substrate
    C_x = constants['C_x']['value']    # Carbon content in biomass
    mu_set = constants['mu_set']['value'] # set specific growth rate
    Kp = constants['Kp']['value']      # static gain of the sensor given as the inverse of the probe response time

    '''------------------- feed options  -------------------'''



    #use S_feed as your feed concentration in your Formulas
    S_feed = inputs['Feed']['concentration']

    if feed.batch:
        pass # put here the things that should happen while we dont feed
    if feed.feeding_first:
        pass # put here the things that should happen when feeding starts (will happen only once)
        # delete this if not needed but remember!: you can call with feed.ts the time when the feed started and with feed.X_fedbatch_start the Biomass at the moment of feed start
    if feed.feeding_exp:
        pass # put here something that should happen while we feed
    if feed.feeding_const:
        pass # put here something that should happen while we induce

    '''------------------- put here your equations  -------------------'''

    q_s = (q_s_max / (1 + (A / K_ia)) * S / (S + K_s)) # put here your equation to calculate q_s (specific substrate uptake rate)
    q_s_of = p_a_max * (q_s / (q_s + K_ap)) # put here your equation to calculate q_s_of (specific substrate uptake rate for overflow)
    q_s_ox = (q_s - q_s_of) * (DOTa / (DOTa + K_o)) # put here your equation to calculate q_s_ox (specific substrate uptake rate for oxygen)
    q_s_an = (q_s_ox - q_m) * Y_em * C_x / C_s
    p_a = q_s_of * Y_as # put here your equation to calculate p_a (specific acetate production rate)
    q_sa = (q_a_max / (1 + (q_s / K_is))) * (A / (A + K_sa)) # put here your equation to calculate q_sa (specific acetate consumption rate)
    q_a = p_a - q_sa # put here your equation to calculate q_a (specific acetate balance rate)
    q_o = (q_s_ox - q_s_an) * Y_os + q_sa * Y_oa # put here your equation to calculate q_o (specific oxygen uptake rate)
    my = (q_s_ox - q_m) * Y_em + q_s_of * Y_xsof + q_sa * Y_xa # put here your equation to calculate my (specific growth rate)

    # remeber to add the variables you want to plot and need for your differential equations that are not rates
    return {
    'q_s': q_s,    'q_s_of': q_s_of,    'q_s_ox': q_s_ox, 'q_s_an': q_s_an,    'p_a': p_a,    'q_sa': q_sa,
    'q_a': q_a,    'q_o': q_o,    'my': my,
           }

In [15]:
# Function for the hole Model
def dif_simple(x, t):




    '''------------------- generated variables -------------------'''

    X = x[0]    # Biomass
    S = x[1]    # extracellular concentration of Substrate
    A = x[2]    # Acetate
    V = x[3]    # Volume of the Medium in the Reactor
    DOTa = x[4]    # actual dissolved oxygen tension
    DOT = x[5]    # measured dissolved oxygen tension

    '''------------------- generated parameters -------------------'''

    K_ap = parameters['K_ap']['value']    # Monod-type saturation constant, intracellular acetate production
    K_sa = parameters['K_sa']['value']    # affinity constant, acetate consumption
    K_o = parameters['K_o']['value']    # affinity constant, oxygen consumption
    K_s = parameters['K_s']['value']    # affinity constant, substrate consumption
    K_ia = parameters['K_ia']['value']    # inhibition constant, inhibition of glucose uptake by acetate
    K_is = parameters['K_is']['value']    # inhibition constant, inhibition of acetate uptake by glucose
    p_a_max = parameters['p_a_max']['value']    # max specific acetate production rate
    q_a_max = parameters['q_a_max']['value']    # max specific acetate consumption rate
    q_m = parameters['q_m']['value']    # specific maintenance coefficient
    q_s_max = parameters['q_s_max']['value']    # max spec glucose uptake rate
    Y_as = parameters['Y_as']['value']    # Yield coefficient of Acetate on substrate
    Y_oa = parameters['Y_oa']['value']    # Yield coefficient of oxygen on Acetate
    Y_xa = parameters['Y_xa']['value']    # Yield coefficient of biomass on Acetate
    Y_em = parameters['Y_em']['value']    # Yield coefficient of biomass exclusive of maintenance
    Y_os = parameters['Y_os']['value']    # Yield coefficient of oxygen on Substrate
    Y_xsof = parameters['Y_xsof']['value']    # yield of biomass on substrate from auxiliary overflow routes, such as the mixed acid and pentose-phosphate pathways

    '''------------------- generated constants -------------------'''

    K_LA = constants['K_LA']['value']    # volumetric mass transfer coefficient
    DOT_star = constants['DOT_star']['value']    # saturation value of dissolved oxygen in the medium
    H = constants['H']['value']    # Henry’s law coefficient
    tau = constants['tau']['value']    # probe response time
    C_s = constants['C_s']['value']    # Carbon content in substrate
    C_x = constants['C_x']['value']    # Carbon content in biomass
    mu_set = constants['mu_set']['value'] # set specific growth rate
    Kp = constants['Kp']['value']      # static gain of the sensor given as the inverse of the probe response time

    rs = algebra(x, t)

    '''------------------- generated rates -------------------'''

    q_s = rs['q_s']    #specific substrate uptake rate
    q_s_of = rs['q_s_of']    #specific substrate uptake rate for overflow
    q_s_ox = rs['q_s_ox']    #specific substrate uptake rate for oxygen
    p_a = rs['p_a']    #specific acetate production rate
    q_sa = rs['q_sa']    #specific acetate consumption rate
    q_a = rs['q_a']    #specific acetate balance rate
    q_o = rs['q_o']    #specific oxygen uptake rate
    my = rs['my']    #specific growth rate


    '''------------------- feed options  -------------------'''

    feed.get(t, S, X)    # S and X are the variables for Substrate and Biomass change them if needed

    F0 = 0.145*60/1000 # beginning of feed

    #use S_feed as your feed concentration in your Formulas
    S_feed = inputs['Feed']['concentration']

    V0 = variables['Volume']['initial_value'] # import V0 as initial volume
    # this is an optional feature you can  use if you want some changes depending on the feeding state
    if feed.batch:
        # put here the things that should happen while we dont feed
        F = 0
    if feed.feeding_first:
        # put here the things that should happen when feeding starts (will happen only once)
        pass # delete this if not needed but remember!: you can call with feed.ts the time when the feed started and with feed.X_fedbatch_start the Biomass at the moment of feed start
    if feed.feeding_exp:
        F = F0 * exp(mu_set * (t - feed.ts))
    if feed.feeding_const:
        F = 0.188 #put here your induction equation(s)
    '''------------------- put here your differential equations -------------------'''

    dXdt = -X * F / V + my * X #write here the right hand side of the differential equation of X
    dSdt = F / V * (S_feed - S) - q_s * X #write here the right hand side of the differential equation of S
    dAdt = - A * F / V + q_a * X #write here the right hand side of the differential equation of A
    dVdt = F #write here the right hand side of the differential equation of V
    dDOTadt = K_LA * (DOT_star - DOTa) - q_o * H * X #write here the right hand side of the differential equation of DOTa
    dDOTdt = Kp * (DOTa - DOT) #write here the right hand side of the differential equation of DOT

    #this will be used by odeint
    dxdt = [dXdt, dSdt, dAdt, dVdt, dDOTadt, dDOTdt]
    return dxdt

In [16]:
# Define initial conditions and make a test run to estimate the feed start
x0 = [variables['Biomass']['initial_value'], variables['Substrate']['initial_value'], variables['Acetate']['initial_value'],variables['Volume']['initial_value'], variables['DOTa']['initial_value'], variables['DOT']['initial_value']]

x = odeint(dif_simple, x0, t)
# Print the feed start time once
print('Feed started at:', feed.ts)

Feed started at: None


In [17]:
# Actual integration phase by phase for more smoothness

if feed.t_const is None and feed.t_fedbatch_start is None and feed.substrate_limit is None:
    mode = 'Mode: Batch only'
elif feed.t_const is not None and feed.t_fedbatch_start is None and feed.substrate_limit is None:
    t_1 = np.linspace(t0, t_constant_feed, int(t_constant_feed/dt)+1)
    t_2 = np.linspace(t_constant_feed, t_end, int((t_end - t_constant_feed)/dt)+1)
    feed = cfb.Feed(t_const = t_constant_feed)
    x_1 = odeint(dif_simple, x0, t_1)
    x_2 = odeint(dif_simple, x_1[-1], t_2)
    x = np.vstack((x_1[:-1], x_2))
    mode = 'Mode: Constant feed only'
elif feed.t_const is None and feed.t_fedbatch_start is not None and feed.substrate_limit is None:
    t_1 = np.linspace(t0, t_fedbatch_start, int(t_fedbatch_start/dt)+1)
    t_2 = np.linspace(t_fedbatch_start, t_end, int((t_end - t_fedbatch_start)/dt)+1)
    feed = cfb.Feed(t_fedbatch_start = t_fedbatch_start)
    x_1 = odeint(dif_simple, x0, t_1)
    x_2 = odeint(dif_simple, x_1[-1], t_2)
    x = np.vstack((x_1[:-1], x_2))
    mode = 'Mode: Fedbatch only (start by time)'
elif feed.t_const is None and feed.t_fedbatch_start is None and feed.substrate_limit is not None:
    t_1 = np.linspace(t0, feed.ts, int(feed.ts/dt)+1)
    t_2 = np.linspace(feed.ts, t_end, int((t_end - feed.ts)/dt)+1)
    feed = cfb.Feed(substrate_limit = substrate_limit_fedbatch)
    x_1 = odeint(dif_simple, x0, t_1)
    x_2 = odeint(dif_simple, x_1[-1], t_2)
    x = np.vstack((x_1[:-1], x_2))
    mode = 'Mode: Fedbatch only (start by substrate limit)'
elif feed.t_const is not None and feed.t_fedbatch_start is not None and feed.substrate_limit is None:
    t_1 = np.linspace(t0, t_fedbatch_start, int(t_fedbatch_start/dt)+1)
    t_2 = np.linspace(t_fedbatch_start, t_constant_feed, int((t_constant_feed - t_fedbatch_start)/dt)+1)
    t_3 = np.linspace(t_constant_feed, t_end, int((t_end - t_constant_feed)/dt)+1)
    feed = cfb.Feed(t_const = t_constant_feed, t_fedbatch_start = t_fedbatch_start)
    x_1 = odeint(dif_simple, x0, t_1)
    x_2 = odeint(dif_simple, x_1[-1], t_2)
    x_3 = odeint(dif_simple, x_2[-1], t_3)
    x = np.vstack((x_1[:-1], x_2[:-1], x_3))
    mode = 'Mode: Const. feed + fedbatch (by time)'
elif feed.t_const is not None and feed.t_fedbatch_start is None and feed.substrate_limit is not None:
    t_1 = np.linspace(t0, feed.ts, int(feed.ts/dt)+1)
    t_2 = np.linspace(feed.ts, t_constant_feed, int((t_constant_feed - feed.ts)/dt)+1)
    t_3 = np.linspace(t_constant_feed, t_end, int((t_end - t_constant_feed)/dt)+1)
    feed = cfb.Feed(t_const = t_constant_feed, substrate_limit = substrate_limit_fedbatch)
    x_1 = odeint(dif_simple, x0, t_1)
    x_2 = odeint(dif_simple, x_1[-1], t_2)
    x_3 = odeint(dif_simple, x_2[-1], t_3)
    x = np.vstack((x_1[:-1], x_2[:-1], x_3))
    mode = 'Mode: Const. feed + fedbatch (by substrate)'
elif feed.t_const is None and feed.t_fedbatch_start is not None and feed.substrate_limit is not None:
    t_1 = np.linspace(t0, feed.ts, int(feed.ts/dt)+1)
    t_2 = np.linspace(feed.ts, t_end, int((t_end - feed.ts)/dt)+1)
    feed = cfb.Feed(t_fedbatch_start = t_fedbatch_start, substrate_limit = substrate_limit_fedbatch)
    x_1 = odeint(dif_simple, x0, t_1)
    x_2 = odeint(dif_simple, x_1[-1], t_2)
    x = np.vstack((x_1[:-1], x_2))
    mode = 'Mode: Fedbatch (start by time or substrate)'
elif feed.t_const is not None and feed.t_fedbatch_start is not None and feed.substrate_limit is not None:
    t_1 = np.linspace(t0, feed.ts, int(feed.ts/dt)+1)
    t_2 = np.linspace(feed.ts, t_constant_feed, int((t_constant_feed - feed.ts)/dt)+1)
    t_3 = np.linspace(t_constant_feed, t_end, int((t_end - t_constant_feed)/dt)+1)
    feed = cfb.Feed(t_const = t_constant_feed, t_fedbatch_start = t_fedbatch_start, substrate_limit = substrate_limit_fedbatch)
    x_1 = odeint(dif_simple, x0, t_1)
    x_2 = odeint(dif_simple, x_1[-1], t_2)
    x_3 = odeint(dif_simple, x_2[-1], t_3)
    x = np.vstack((x_1[:-1], x_2[:-1], x_3))
    mode = 'Mode: Const. feed + fedbatch (start by time or substrate)'
print(mode)

Mode: Batch only


In [18]:
'''Here all the metabolic fluxes that will be plotet wil be added to x '''
algebra_results = []    #just to initiate the variable

# Iterate over time steps to access individual values
for state_vector, time_point in zip(x, t):
    #if you calculate the oxygen consumption rate calculate it here, because its not a differential equation
    X, S, A, V, DOTa, DOT = state_vector
    results = algebra(state_vector, time_point)
    algebra_results.append([
        results['q_s'],
        results['q_s_of'],
        results['q_s_ox'],
        results['q_s_an'],
        results['p_a'],
        results['q_sa'],
        results['q_a'],
        results['q_o'],
        results['my'],
    ])

#combine everything in x
algebra_array = np.array(algebra_results)
x_and_rs = np.hstack((x, algebra_array))

In [19]:
import pandas as pd
# Load the CSV
df = pd.read_csv("Y_data.csv", header=None)
# Set column names (edit as needed!)
df.columns = ['Volume', 'Biomass', 'Substrate', 'Acetate', 'DOTa', 'DOT', 'Feed']
columns = ['Volume', 'Biomass', 'Substrate', 'Acetate', 'DOTa', 'DOT', 'Feed']

T = np.loadtxt('T_data.csv', delimiter=',')
df.insert(0, 'Time', T)



# Show preview
print(df.head())
# Plot the Paper results
# Split time and values
d_t = df['Time'].to_numpy() - df['Time'].to_numpy()[0] # for starting at 0 hours
data = df.drop(columns=['Time']).to_numpy()
d_t

       Time  Volume   Biomass  Substrate   Acetate       DOTa        DOT  Feed
0  1.999722       2  0.170000   4.940000  0.012900  98.000000  98.000000     0
1  2.002500       2  0.170147   4.939705  0.012933  97.034876  97.837495     0
2  2.005278       2  0.170295   4.939410  0.012966  96.445658  97.562471     0
3  2.008056       2  0.170442   4.939115  0.012998  96.117759  97.246396     0
4  2.010833       2  0.170590   4.938820  0.013031  95.942873  96.944028     0


array([0.00000000e+00, 2.77777778e-03, 5.55555556e-03, ...,
       9.44166667e+00, 9.44472222e+00, 9.44722222e+00])

In [20]:
# Initializing the allthedata dictionary

if feed.t_const is None and feed.t_fedbatch_start is None and feed.substrate_limit is None:
    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t)
elif feed.t_const is not None and feed.t_fedbatch_start is None and feed.substrate_limit is None:
    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_constant_feed = t_2)
elif feed.t_const is None and feed.t_fedbatch_start is not None and feed.substrate_limit is None:
    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2)
elif feed.t_const is None and feed.t_fedbatch_start is None and feed.substrate_limit is not None:
    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2)
elif feed.t_const is not None and feed.t_fedbatch_start is not None and feed.substrate_limit is None:
    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2, t_3)
elif feed.t_const is not None and feed.t_fedbatch_start is None and feed.substrate_limit is not None:
    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2, t_3)
elif feed.t_const is None and feed.t_fedbatch_start is not None and feed.substrate_limit is not None:
    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2)
elif feed.t_const is not None and feed.t_fedbatch_start is not None and feed.substrate_limit is not None:
    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2, t_3)

allthedata = builder.build(data = data, columns = columns, t_data = d_t)

✅ Data successfully aligned with variable order.
⚠️ Warning: 'q_s' (rate) not found in data — filled with NaN.
⚠️ Warning: 'q_s_of' (rate) not found in data — filled with NaN.
⚠️ Warning: 'q_s_ox' (rate) not found in data — filled with NaN.
⚠️ Warning: 'q_s_ox_an' (rate) not found in data — filled with NaN.
⚠️ Warning: 'p_a' (rate) not found in data — filled with NaN.
⚠️ Warning: 'q_sa' (rate) not found in data — filled with NaN.
⚠️ Warning: 'q_a' (rate) not found in data — filled with NaN.
⚠️ Warning: 'q_o' (rate) not found in data — filled with NaN.
⚠️ Warning: 'my' (rate) not found in data — filled with NaN.


In [21]:
path_to_results, datestring, run_id = builder.init_saving(1, 1, os.path.join(os.getcwdb().decode('utf-8'), 'Anane_results'), None)
allthedata_plot = {"1": allthedata}
plot = cplt.PlotPlotly(
    allthedata_plot,
    path_to_results,
    plot_info=f"{datestring}_{run_id}",
    plot_mode="white_w_grid",
    colorscheme="matlab_default",
    plot_formats=("html")
)
plot.use_column_title = True
plot.plot_height_multiple_rows = 1000  # default is 200, increase for bigger plots
plot.plot_height_one_row = 600  # default is 225
plot.plot_by_expID(
    plot_replicates=False,
    show_legend=False,
    plot_SE_data=False,
    split_at_feed_start=False,
    range_end=None,
    plot_DOT_actual = True, # important for plotting DOTa
    plot_multiple_list=[
        dict(members=["Biomass", "Substrate"], axes=["left", "right"])
    ]
)

Results are saved here: F:\Bachelorarbeit\git-tu\e-coli-model-overview\Library_of_models\ecoli_2017_Anane_Acetate_cycling\Paper_comparison\Anane_results\20251021_23-23-42_1
